In [2]:
import pandas as pd
import numpy as np

In [38]:
import chardet

with open('C:/Users/lenov/Documents/Template_MLOps_accidents/data/raw/caracteristiques-2021.csv', 'rb') as file:
    result = chardet.detect(file.read())

print(result['encoding'])

ModuleNotFoundError: No module named 'chardet'

In [39]:
#Création des DF
df_users=pd.read_csv("C:/Users/lenov/Documents/Template_MLOps_accidents/data/raw/usagers-2021.csv", sep=";")
df_places=pd.read_csv("C:/Users/lenov/Documents/Template_MLOps_accidents/data/raw/lieux-2021.csv", sep=";", header=0, low_memory=False)
df_caract=pd.read_csv("C:/Users/lenov/Documents/Template_MLOps_accidents/data/raw/caracteristiques-2021.csv", sep = ";", encoding='utf-8')
df_veh=pd.read_csv("C:/Users/lenov/Documents/Template_MLOps_accidents/data/raw/vehicules-2021.csv", sep=";")

In [26]:
df = pd.read_csv("C:/Users/lenov/Desktop/caracteristics.csv",encoding='ISO-8859-1')

C:\Users\lenov\AppData\Local\Temp\ipykernel_11392\574975583.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/lenov/Desktop/caracteristics.csv",encoding='ISO-8859-1')


# DataSet Users 

In [6]:
df_users.isna().sum()

Num_Acc           0
id_vehicule       0
num_veh           0
place             0
catu              0
grav              0
sexe              0
an_nais        3067
trajet            0
secu1             0
secu2             0
secu3             0
locp              0
actp              0
etatp             0
dtype: int64

In [7]:
df_places.isna().sum()

Num_Acc        0
catr           0
voie        4407
v1             0
v2         51528
circ           0
nbv            0
vosp           0
prof           0
pr             0
pr1            0
plan           0
lartpc     56410
larrout        0
surf           0
infra          0
situ           0
vma            0
dtype: int64

# Dataset caractéristiques

In [40]:
df_caract.head()

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202100000001,30,11,2021,07:32,2,30,30319,1,1,1,1,CD 981,"44,0389580000","4,3480220000"
1,202100000002,25,9,2021,14:20,1,51,51544,1,3,1,3,Aire de repos croisement D20E9 aprÃ¨s sortie D977,"49,2421290000","4,5545460000"
2,202100000003,15,7,2021,07:55,1,85,85048,2,1,7,6,15 rue FranÃ§ois Nicolas,"46,9219500000","-0,9644600000"
3,202100000004,27,3,2021,19:45,5,93,93005,2,2,3,6,Route de Mitry,"48,9493634583","2,5196639908"
4,202100000005,25,2,2021,07:20,5,76,76429,2,1,1,2,PARIS. ROUTE DE,"49,4083800000","1,1458100000"


Suppression variable adresse car trop peu utilisable

In [41]:
df_caract = df_caract.drop(columns = 'adr')

In [47]:
df_caract.shape

(56518, 14)

8 lines have value -1 for the variable atm 

In [50]:
df_caract = df_caract[df_caract["atm"]!=-1]

Grouping the modalities of the atm ( Atmosheric Conditions) variable into Risky and Normal. We include Other in Normal.

In [51]:
df_caract['atm'].unique()

array([1, 7, 3, 2, 8, 9, 5, 4, 6], dtype=int64)

In [57]:
dico = {1:'Normal', 2:'Risky', 3:'Risky', 4:'Risky', 5:'Risky', 6:'Risky',7:'Risky', 8:'Normal', 9:'Normal'}
df_caract["atm"] = df_caract["atm"].replace(dico)
df_caract.head()

C:\Users\lenov\AppData\Local\Temp\ipykernel_11392\1185451846.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_caract["atm"] = df_caract["atm"].replace(dico)


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,lat,long
0,202100000001,30,11,2021,07:32,2,30,30319,1,1,Normal,1,"44,0389580000","4,3480220000"
1,202100000002,25,9,2021,14:20,1,51,51544,1,3,Normal,3,"49,2421290000","4,5545460000"
2,202100000003,15,7,2021,07:55,1,85,85048,2,1,Risky,6,"46,9219500000","-0,9644600000"
3,202100000004,27,3,2021,19:45,5,93,93005,2,2,Risky,6,"48,9493634583","2,5196639908"
4,202100000005,25,2,2021,07:20,5,76,76429,2,1,Normal,2,"49,4083800000","1,1458100000"


In [52]:
df_caract.isna().sum()

Num_Acc    0
jour       0
mois       0
an         0
hrmn       0
lum        0
dep        0
com        0
agg        0
int        0
atm        0
col        0
lat        0
long       0
dtype: int64

In [54]:
df_caract[df_caract["atm"]==9]

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,lat,long
39,202100000040,12,11,2021,21:25,5,987,98736,2,9,9,3,"-17,5279960000","-149,5482580000"
332,202100000333,22,11,2021,22:15,3,50,50502,1,1,9,2,"49,0976000000","-1,0605000000"
547,202100000548,16,12,2021,12:40,1,73,73212,2,1,9,4,"45,5433702918","6,3019246459"
806,202100000807,31,12,2021,19:30,3,33,33501,1,1,9,6,"44,6521040000","-0,6169030000"
834,202100000835,14,12,2021,17:00,1,05,05061,2,1,9,6,"44,5639600000","6,0837900000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56379,202100056380,2,1,2021,20:30,3,91,91471,1,1,9,4,"48,7141000000","2,1756800000"
56454,202100056455,1,1,2021,19:05,3,80,80825,1,1,9,6,"49,9594075089","1,8430694944"
56464,202100056465,3,1,2021,11:30,1,64,64498,1,2,9,7,"43,1454570000","-0,1490310000"
56491,202100056492,1,1,2021,22:00,3,93,93007,1,1,9,6,"48,9550900000","2,4575400000"


In [ ]:
dico = {1:'Normal', 2:'Risky', 3:'Risky', 4:'Risky', 5:'Risky', 6:'Risky',7:'Risky', 8:'Normal', 9:'Normal'}

In [53]:
df_veh.isna().sum()

Num_Acc            0
id_vehicule        0
num_veh            0
senc               0
catv               0
obs                0
obsm               0
choc               0
manv               0
motor              0
occutc         96571
dtype: int64